#**Unzipping Data**

In [2]:
import glob
import os, zipfile
RAW_DIR = "/content/raw"
os.makedirs(RAW_DIR, exist_ok=True)

zip_path = "/content/byggnad_gpkg.zip"  # <-- change if your filename differs

with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(RAW_DIR)

print("Unzipped into:", RAW_DIR)

Unzipped into: /content/raw


In [3]:
for root, dirs, files in os.walk(RAW_DIR):
    for f in files:
        if f.lower().endswith(".gpkg"):
            print("Found GPKG:", os.path.join(root, f))

Found GPKG: /content/raw/anlaggningsomrade_sverige.gpkg
Found GPKG: /content/raw/byggnad_sverige.gpkg


##**Imports**

In [57]:
import geopandas as gpd
import fiona
import json
import numpy as np
from shapely.geometry import box
import laspy
from pathlib import Path
from shapely.geometry import Point
import pandas as pd
import rasterio
from rasterio.mask import mask
import imageio.v3 as iio

In [7]:
gpkg_path = "/content/raw/byggnad_sverige.gpkg"

# List layers
layers = list(fiona.listlayers(gpkg_path))
print("Layers inside byggnad_sverige.gpkg:")
print(layers)

Layers inside byggnad_sverige.gpkg:
['byggnad']


In [8]:
layer_name = layers[0]

gdf = gpd.read_file(gpkg_path, layer=layer_name)

print("Rows:", len(gdf))
print("CRS:", gdf.crs)
print("Bounds:", gdf.total_bounds)
print("Geometry types:", gdf.geom_type.value_counts())
print("Columns:", list(gdf.columns))

gdf.head(3)

Rows: 3760
CRS: EPSG:3006
Bounds: [ 319988. 6397230.  321351. 6401624.]
Geometry types: MultiPolygon    3646
Polygon          114
Name: count, dtype: int64
Columns: ['objektidentitet', 'versiongiltigfran', 'lagesosakerhetplan', 'lagesosakerhethojd', 'ursprunglig_organisation', 'objektversion', 'objekttypnr', 'objekttyp', 'insamlingslage', 'byggnadsnamn1', 'byggnadsnamn2', 'byggnadsnamn3', 'husnummer', 'huvudbyggnad', 'andamal1', 'andamal2', 'andamal3', 'andamal4', 'andamal5', 'geometry']


,objektidentitet,versiongiltigfran,lagesosakerhetplan,lagesosakerhethojd,ursprunglig_organisation,objektversion,objekttypnr,objekttyp,insamlingslage,byggnadsnamn1,byggnadsnamn2,byggnadsnamn3,husnummer,huvudbyggnad,andamal1,andamal2,andamal3,andamal4,andamal5,geometry
0,6924efe8-d435-4bfb-bdaf-bb7f18576b12,2011-03-23 09:09:40+00:00,0.060,2.5,Lantmäteriet,1,2061,Bostad,Takkant,None,None,None,1,Nej,Bostad;Flerfamiljshus,Verksamhet;,None,None,None,"MULTIPOLYGON (((320812.738 6400637.806, 320813..."
1,cfd2cc69-69a7-486b-9641-518b62dd489a,2011-03-23 09:09:40+00:00,0.025,2.5,Lantmäteriet,1,2066,Komplementbyggnad,Fasad,None,None,None,4,Nej,Komplementbyggnad;,None,None,None,None,"MULTIPOLYGON (((321122.706 6400888.63, 321183...."
2,88827b9a-90d8-40a5-b6e9-1b7ac2695988,2024-04-23 12:36:47+00:00,0.100,NaN,Göteborg,3,2061,Bostad,Takkant,None,None,None,2,Nej,Bostad;Flerfamiljshus,None,None,None,None,"MULTIPOLYGON (((320616.181 6397789.803, 320626..."


**Small Geometry Exploration**

In [9]:
# Take first building
b = gdf.iloc[0]

print("Geometry type:", b.geometry.geom_type)

if b.geometry.geom_type == "MultiPolygon":
    print("Number of polygons inside:", len(b.geometry.geoms))

# Show coordinates of first polygon
poly = b.geometry.geoms[0] if b.geometry.geom_type == "MultiPolygon" else b.geometry
print("Number of exterior points:", len(poly.exterior.coords))

list(poly.exterior.coords)[:5]

Geometry type: MultiPolygon
Number of polygons inside: 1
Number of exterior points: 9


[(320812.738, 6400637.806),
 (320813.022, 6400637.614),
 (320813.385, 6400638.159),
 (320828.168, 6400628.309),
 (320828.352, 6400628.685)]

**Derive bbox + origin from the buildings dataset**

In [13]:
# 1) Get bbox from the buildings dataset
xmin, ymin, xmax, ymax = gdf.total_bounds

# 2) Compute origin as center of bbox
origin_e = float((xmin + xmax) / 2)
origin_n = float((ymin + ymax) / 2)

# 3) Compute size
width_m  = float(xmax - xmin)
height_m = float(ymax - ymin)

study_area = {
    "crs_epsg": 3006,
    "bbox": {"xmin": float(xmin), "ymin": float(ymin), "xmax": float(xmax), "ymax": float(ymax)},
    "origin_sweref99tm": {"easting": origin_e, "northing": origin_n},
    "size_m": {"width": width_m, "height": height_m}
}

print(json.dumps(study_area, indent=2))

# Save for later steps
out_path = "/content/study_area.json"
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(study_area, f, indent=2)

print("\nSaved:", out_path)

{
  "crs_epsg": 3006,
  "bbox": {
    "xmin": 319988.0,
    "ymin": 6397230.0,
    "xmax": 321351.0,
    "ymax": 6401624.0
  },
  "origin_sweref99tm": {
    "easting": 320669.5,
    "northing": 6399427.0
  },
  "size_m": {
    "width": 1363.0,
    "height": 4394.0
  }
}

Saved: /content/study_area.json


**Create a 1×1 km box centered at your origin**

In [16]:
PROC_DIR = "/content/processed"
os.makedirs(PROC_DIR, exist_ok=True)

out_path = os.path.join(PROC_DIR, "buildings_clipped_1km.geojson")
gdf_1km.to_file(out_path, driver="GeoJSON")
print("Saved:", out_path)

Saved: /content/processed/buildings_clipped_1km.geojson


In [17]:
# Load the frozen study area info
import json
with open("/content/study_area.json") as f:
    study = json.load(f)

E0 = study["origin_sweref99tm"]["easting"]
N0 = study["origin_sweref99tm"]["northing"]

HALF = 500  # 1000m x 1000m

bbox_1km = box(E0 - HALF, N0 - HALF, E0 + HALF, N0 + HALF)
bbox_gdf = gpd.GeoDataFrame({"geometry":[bbox_1km]}, crs="EPSG:3006")

# Clip buildings
gdf_1km = gpd.clip(gdf, bbox_gdf)

print("Buildings in 1x1 km box:", len(gdf_1km))
print("Bounds:", gdf_1km.total_bounds)

out_path = "/content/processed/buildings_clipped_1km.geojson"
gdf_1km.to_file(out_path, driver="GeoJSON")
print("Saved:", out_path)

Buildings in 1x1 km box: 701
Bounds: [ 320169.5 6398927.   321169.5 6399927. ]
Saved: /content/processed/buildings_clipped_1km.geojson


In [18]:
check = gpd.read_file("/content/processed/buildings_clipped_1km.geojson")
print("Read back rows:", len(check))
print("CRS:", check.crs)
print("Bounds:", check.total_bounds)

Read back rows: 701
CRS: EPSG:3006
Bounds: [ 320169.5 6398927.   321169.5 6399927. ]


#**Working on LiDAR**

In [25]:
lidar_zip = "/content/laserdata_nh.zip"

with zipfile.ZipFile(lidar_zip, "r") as z:
    z.extractall("/content/raw")

print("LiDAR extracted.")

LiDAR extracted.


In [26]:
laz_files = sorted(glob.glob("/content/raw/**/*.laz", recursive=True))

print("Found LAZ files:", len(laz_files))
for p in laz_files[:5]:
    print(p)

Found LAZ files: 6
/content/raw/09B002_639_31_5075.laz
/content/raw/09B002_639_31_7575.laz
/content/raw/09B002_639_32_5000.laz
/content/raw/09B002_639_32_7500.laz
/content/raw/09B008_640_31_0075.laz


In [28]:
tile_path = "/content/raw/09B002_639_31_5075.laz"  # pick the first one

las = laspy.read(tile_path)

print("Tile:", tile_path)
print("Point count:", len(las.x))

# Coordinate ranges
print("\nX range:", float(np.min(las.x)), "to", float(np.max(las.x)))
print("Y range:", float(np.min(las.y)), "to", float(np.max(las.y)))
print("Z range:", float(np.min(las.z)), "to", float(np.max(las.z)))

# What attributes exist for each point
print("\nPoint dimensions available:")
print(las.point_format.dimension_names)

# Classification is critical for building heights
if "classification" in las.point_format.dimension_names:
    cls, counts = np.unique(las.classification, return_counts=True)
    print("\nClassification summary (first 20 classes):")
    for c, n in list(zip(cls, counts))[:20]:
        print(f"class {int(c):>3}: {int(n)} points")
else:
    print("\nNo 'classification' dimension found in this LAZ file.")

Tile: /content/raw/09B002_639_31_5075.laz
Point count: 10144103

X range: 317500.0 to 319999.99
Y range: 6395000.0 to 6397499.99
Z range: 10.26 to 177.49

Point dimensions available:
<generator object PointFormat.dimension_names.<locals>.<genexpr> at 0x7ad97c148f40>

Classification summary (first 20 classes):
class   1: 5054604 points
class   2: 5071314 points
class   9: 16157 points
class  11: 2028 points


In [31]:
# Study area bounds
xmin, ymin, xmax, ymax = gdf_1km.total_bounds

print("Study area bounds:")
print("X:", xmin, "to", xmax)
print("Y:", ymin, "to", ymax)

laz_files = sorted(glob.glob("/content/raw/*.laz"))

print("\nChecking tiles for overlap...\n")

overlapping_tiles = []

for tile_path in laz_files:
    with laspy.open(tile_path) as f:
        header = f.header

        tile_xmin = header.mins[0]
        tile_ymin = header.mins[1]
        tile_xmax = header.maxs[0]
        tile_ymax = header.maxs[1]

    # Bounding box intersection logic
    intersects = not (
        tile_xmax < xmin or
        tile_xmin > xmax or
        tile_ymax < ymin or
        tile_ymin > ymax
    )

    print(tile_path)
    print("  Tile X:", tile_xmin, "to", tile_xmax)
    print("  Tile Y:", tile_ymin, "to", tile_ymax)
    print("  Overlaps study area?", intersects)
    print()

    if intersects:
        overlapping_tiles.append(tile_path)

print("Overlapping tiles:")
for t in overlapping_tiles:
    print("  ", t)

Study area bounds:
X: 320169.5 to 321169.5
Y: 6398927.0 to 6399927.0

Checking tiles for overlap...

/content/raw/09B002_639_31_5075.laz
  Tile X: 317500.0 to 319999.99
  Tile Y: 6395000.0 to 6397499.99
  Overlaps study area? False

/content/raw/09B002_639_31_7575.laz
  Tile X: 317500.0 to 319999.99
  Tile Y: 6397500.0 to 6399999.99
  Overlaps study area? False

/content/raw/09B002_639_32_5000.laz
  Tile X: 320000.0 to 322499.99
  Tile Y: 6395000.0 to 6397499.99
  Overlaps study area? False

/content/raw/09B002_639_32_7500.laz
  Tile X: 320000.0 to 322499.99
  Tile Y: 6397500.0 to 6399999.99
  Overlaps study area? True

/content/raw/09B008_640_31_0075.laz
  Tile X: 317500.0 to 319999.99
  Tile Y: 6400000.0 to 6402499.99
  Overlaps study area? False

/content/raw/09B008_640_32_0000.laz
  Tile X: 320000.0 to 322499.99
  Tile Y: 6400000.0 to 6402499.99
  Overlaps study area? False

Overlapping tiles:
   /content/raw/09B002_639_32_7500.laz


In [32]:
tile_path = "/content/raw/09B002_639_32_7500.laz"
xmin, ymin, xmax, ymax = gdf_1km.total_bounds

las = laspy.read(tile_path)

x = las.x
y = las.y
z = las.z

mask = (x >= xmin) & (x <= xmax) & (y >= ymin) & (y <= ymax)

print("Total points in tile:", len(x))
print("Points inside study bbox:", int(mask.sum()))

# Extract filtered arrays
xf = x[mask]
yf = y[mask]
zf = z[mask]

# Optional: classification if present
if "classification" in las.point_format.dimension_names:
    cf = las.classification[mask]
    cls, counts = np.unique(cf, return_counts=True)
    print("\nClasses inside bbox:")
    for c, n in zip(cls, counts):
        print(int(c), int(n))
else:
    cf = None
    print("\nNo classification field available.")

Total points in tile: 9215428
Points inside study bbox: 1177698

Classes inside bbox:
1 588109
2 582162
9 199
11 7228


In [38]:
# Use the filtered arrays you already created: xf, yf, zf, cf
# If they are not in memory, re-run the previous filtering cell first.

# Convert laspy arrays to real numpy arrays first
zf_np = np.asarray(zf, dtype=np.float32)
cf_np = np.asarray(cf, dtype=np.uint8)

df_pts = pd.DataFrame({
    "z": zf_np,
    "cls": cf_np
})

# Geometry creation (xf and yf are also ScaledArrayView, so convert them too)
xf_np = np.asarray(xf, dtype=np.float64)
yf_np = np.asarray(yf, dtype=np.float64)

geom = gpd.points_from_xy(xf_np, yf_np)

gdf_pts = gpd.GeoDataFrame(df_pts, geometry=geom, crs="EPSG:3006")

print("Points GDF:", len(gdf_pts))
gdf_pts.head()

Points GDF: 1177698


,z,cls,geometry
0,41.740002,2,POINT (321169.44 6399919.19)
1,41.439999,2,POINT (321169.31 6399920.94)
2,43.709999,1,POINT (321169.35 6399922.32)
3,41.080002,2,POINT (321169.08 6399924.39)
4,40.810001,2,POINT (321168.95 6399926.12)


In [39]:
buildings = gdf_1km[["objektidentitet", "geometry"]].copy()
print("Buildings:", len(buildings))
buildings.head(2)

Buildings: 701


,objektidentitet,geometry
2761,b74c0af5-9e2a-4884-a408-510d18176b91,"MULTIPOLYGON (((320319.376 6398928.116, 320318..."
3241,d76723d9-06cb-4157-8c47-5a4a798e6dd0,"POLYGON ((320320.289 6398931.465, 320328.185 6..."


In [40]:
b = buildings.iloc[0]
b_id = b["objektidentitet"]
b_geom = b["geometry"]

print("Building id:", b_id)
print("Geom type:", b_geom.geom_type)
print("Area (m²):", float(b_geom.area))

Building id: b74c0af5-9e2a-4884-a408-510d18176b91
Geom type: MultiPolygon
Area (m²): 29.428099533065566


In [43]:
# Points inside the polygon footprint
inside = gdf_pts[gdf_pts.within(b_geom)]
print("Points inside:", len(inside))
print("Class counts inside:\n", inside["cls"].value_counts().head(10))
print("z min/median/max:", float(inside["z"].min()), float(inside["z"].median()), float(inside["z"].max()))

Points inside: 99
Class counts inside:
 cls
1    99
Name: count, dtype: int64
z min/median/max: 18.649999618530273 35.90999984741211 41.08000183105469


In [42]:
if len(inside) == 0:
    print("No LiDAR points inside this building. Try a different building index.")
else:
    roof_z = float(inside["z"].max())

    inside_ground = inside[inside["cls"] == 2]
    if len(inside_ground) > 0:
        ground_z = float(inside_ground["z"].min())
        ground_source = "cls==2"
    else:
        ground_z = float(inside["z"].min())
        ground_source = "any"

    height_m = roof_z - ground_z

    print("roof_z:", roof_z)
    print("ground_z:", ground_z, f"(source: {ground_source})")
    print("height_m:", height_m)

roof_z: 41.08000183105469
ground_z: 18.649999618530273 (source: any)
height_m: 22.430002212524414


**Spatial Join (One Big Operation)**

In [44]:
buildings = gdf_1km[["objektidentitet", "geometry"]].copy()

joined = gpd.sjoin(
    gdf_pts,
    buildings,
    how="inner",
    predicate="within"
)

print("Total point-building matches:", len(joined))
joined.head()

Total point-building matches: 380415


,z,cls,geometry,index_right,objektidentitet
3344,64.820000,2,POINT (321168.36 6399590.82),2099,8deae4fb-73ea-4a80-87a7-73e2d438b4b4
3865,65.099998,2,POINT (321167.61 6399590.81),2099,8deae4fb-73ea-4a80-87a7-73e2d438b4b4
3866,65.150002,2,POINT (321167.74 6399589),2099,8deae4fb-73ea-4a80-87a7-73e2d438b4b4
3868,65.980003,2,POINT (321168.04 6399585.4),835,7931f624-5902-407e-9cc7-f152c6bf11ca
3898,66.400002,2,POINT (321167.37 6399581.31),835,7931f624-5902-407e-9cc7-f152c6bf11ca


**Compute Height for All Buildings**

In [45]:
# Roof = highest z per building
roof = joined.groupby("objektidentitet")["z"].max().rename("roof_z")

# Ground from class 2 if available
ground_cls2 = (
    joined[joined["cls"] == 2]
    .groupby("objektidentitet")["z"]
    .min()
    .rename("ground_z_cls2")
)

# Fallback ground = lowest z of any class
ground_any = joined.groupby("objektidentitet")["z"].min().rename("ground_z_any")

# Combine
heights = pd.concat([roof, ground_cls2, ground_any], axis=1)

# Prefer class 2 ground if present
heights["ground_z"] = heights["ground_z_cls2"].fillna(heights["ground_z_any"])

# Final height
heights["height_m"] = heights["roof_z"] - heights["ground_z"]

print("Height statistics:")
print(heights["height_m"].describe())

print("\nBuildings with computed height:", heights["height_m"].notna().sum())

Height statistics:
count    556.000000
mean      14.787193
std       11.660794
min        0.000000
25%        5.742500
50%       12.675000
75%       22.150000
max       80.979996
Name: height_m, dtype: float64

Buildings with computed height: 556


In [46]:
heights.sort_values("height_m", ascending=False).head(5)

,roof_z,ground_z_cls2,ground_z_any,ground_z,height_m
objektidentitet,,,,,
c7c737ba-c1a5-464a-890e-353fddb87c65,83.699997,2.72,-13.87,2.72,80.979996
8a70d69b-518d-4e79-87b4-d7cf7e2b3d89,81.940002,7.14,6.89,7.14,74.800003
0b29e0e3-f62f-456e-aef8-7e1c5bcd2f89,77.080002,7.14,2.05,7.14,69.940002
5d8544ae-8f8a-4b2e-8137-1f2187adce3b,77.129997,NaN,7.30,7.30,69.829994
2a96f88c-3908-495d-bbd1-1ffacc320a87,67.940002,3.14,3.14,3.14,64.800003


**ttach Heights Back to Buildings**

In [47]:
gdf_with_h = gdf_1km.merge(
    heights[["roof_z", "ground_z", "height_m"]],
    on="objektidentitet",
    how="left"
)

missing = int(gdf_with_h["height_m"].isna().sum())
print("Buildings without LiDAR height:", missing, "out of", len(gdf_with_h))

gdf_with_h.head()

Buildings without LiDAR height: 1 out of 701


,objektidentitet,versiongiltigfran,lagesosakerhetplan,lagesosakerhethojd,ursprunglig_organisation,objektversion,objekttypnr,objekttyp,insamlingslage,byggnadsnamn1,...,huvudbyggnad,andamal1,andamal2,andamal3,andamal4,andamal5,geometry,roof_z,ground_z,height_m
0,b74c0af5-9e2a-4884-a408-510d18176b91,2011-03-23 08:51:32+00:00,0.06,2.5,Lantmäteriet,1,2061,Bostad,Takkant,None,...,Nej,Bostad;Flerfamiljshus,Verksamhet;,None,None,None,"MULTIPOLYGON (((320319.376 6398928.116, 320318...",41.080002,18.65,22.430002
1,d76723d9-06cb-4157-8c47-5a4a798e6dd0,2011-03-23 08:50:27+00:00,0.06,2.5,Lantmäteriet,1,2061,Bostad,Takkant,None,...,Nej,Bostad;Flerfamiljshus,Verksamhet;,None,None,None,"POLYGON ((320320.289 6398931.465, 320328.185 6...",43.340000,7.92,35.419998
2,110f0c2c-6d43-4412-a56d-508aba264d22,2011-03-23 09:08:00+00:00,0.06,2.5,Lantmäteriet,1,2061,Bostad,Takkant,None,...,Nej,Bostad;Flerfamiljshus,Verksamhet;,None,None,None,"POLYGON ((320328.801 6398958.49, 320319.5 6398...",35.939999,8.13,27.809998
3,ae6ccfa2-5cfd-4c38-9a05-001548ba0ea0,2011-03-23 08:54:58+00:00,0.06,2.5,Lantmäteriet,1,2061,Bostad,Takkant,None,...,Nej,Bostad;Flerfamiljshus,Verksamhet;,None,None,None,"POLYGON ((320278.096 6398933.909, 320286.111 6...",43.740002,16.24,27.500002
4,bbe6794b-58df-4cd2-9c71-3ec458184667,2011-03-23 08:51:15+00:00,0.06,2.5,Lantmäteriet,1,2061,Bostad,Takkant,None,...,Nej,Bostad;Flerfamiljshus,Verksamhet;,None,None,None,"POLYGON ((320269.186 6398940.38, 320267.903 63...",44.849998,15.95,28.899998


**Save Final GeoJSON**

In [48]:
PROC_DIR = "/content/processed"
os.makedirs(PROC_DIR, exist_ok=True)

out_path = os.path.join(PROC_DIR, "buildings_with_heights_1km.geojson")
gdf_with_h.to_file(out_path, driver="GeoJSON")

print("Saved:", out_path)

Saved: /content/processed/buildings_with_heights_1km.geojson


# **Unzip the höjddata2m**

In [50]:
RAW_DIR = "/content/raw"
os.makedirs(RAW_DIR, exist_ok=True)

dem_zip = "/content/hojddata2m_clip.zip"

with zipfile.ZipFile(dem_zip, "r") as z:
    z.extractall(RAW_DIR)

print("DEM extracted.")

DEM extracted.


In [51]:
tifs = glob.glob("/content/raw/**/*.tif", recursive=True)
print("Found TIFF files:")
for t in tifs:
    print(t)


Found TIFF files:
/content/raw/hojddata2_mosaik.tif


In [53]:
dem_path = "/content/raw/hojddata2_mosaik.tif"

with rasterio.open(dem_path) as src:
    print("CRS:", src.crs)
    print("Bounds:", src.bounds)
    print("Resolution (m per pixel):", src.res)
    print("Size:", src.width, "x", src.height)
    print("Band count:", src.count)
    print("Dtype:", src.dtypes)
    print("Nodata:", src.nodata)

CRS: EPSG:3006
Bounds: BoundingBox(left=319988.0, bottom=6397230.0, right=321352.0, top=6401624.0)
Resolution (m per pixel): (2.0, 2.0)
Size: 682 x 2197
Band count: 1
Dtype: ('float32',)
Nodata: -9999.0


# **Clip DEM to the 1×1 km box**

In [56]:
# Load the 1x1 km bbox you used for buildings (from gdf_1km bounds)
xmin, ymin, xmax, ymax = gdf_1km.total_bounds
bbox_poly = box(xmin, ymin, xmax, ymax)

dem_path = "/content/raw/hojddata2_mosaik.tif"  # use your DEM path

with rasterio.open(dem_path) as src:
    clipped, clipped_transform = mask(src, [bbox_poly], crop=True)
    dem = clipped[0].astype(np.float32)

    # Convert nodata to NaN
    dem = np.where(dem == src.nodata, np.nan, dem)

print("Clipped DEM shape:", dem.shape)
print("Elevation min/max:", float(np.nanmin(dem)), float(np.nanmax(dem)))

Clipped DEM shape: (501, 501)
Elevation min/max: -0.7489811182022095 69.12937927246094


# **Normalize to 16-bit (0–65535) and save PNG**

In [60]:
PROC_DIR = "/content/processed"
os.makedirs(PROC_DIR, exist_ok=True)

elev_min = float(np.nanmin(dem))
elev_max = float(np.nanmax(dem))

# Normalize to 0..1
norm = (dem - elev_min) / (elev_max - elev_min + 1e-9)
norm = np.clip(norm, 0.0, 1.0)

# Convert to 16-bit
heightmap_16 = (norm * 65535.0).round().astype(np.uint16)

out_hm = os.path.join(PROC_DIR, "heightmap_1km.png")
iio.imwrite(out_hm, heightmap_16)

print("Saved:", out_hm)
print("dtype:", heightmap_16.dtype)
print("value range:", int(heightmap_16.min()), "to", int(heightmap_16.max()))
print("elev_min/max:", elev_min, elev_max)

/tmp/ipython-input-3083554435.py:12: RuntimeWarning: invalid value encountered in cast
  heightmap_16 = (norm * 65535.0).round().astype(np.uint16)


Saved: /content/processed/heightmap_1km.png
dtype: uint16
value range: 0 to 65535
elev_min/max: -0.7489811182022095 69.12937927246094


# **Save elevation range into metadata (for Unity scaling)**

In [61]:
meta_path = "/content/processed/metadata_1km.json"

metadata = {
    "crs_epsg": 3006,
    "bbox": {"xmin": float(xmin), "ymin": float(ymin), "xmax": float(xmax), "ymax": float(ymax)},
    "origin_sweref99tm": {
        "easting": float((xmin + xmax) / 2),
        "northing": float((ymin + ymax) / 2)
    },
    "dem": {
        "elev_min": elev_min,
        "elev_max": elev_max,
        "resolution_m": [2.0, 2.0],
        "shape_px": [int(heightmap_16.shape[0]), int(heightmap_16.shape[1])]
    },
    "outputs": {
        "heightmap_png": "heightmap_1km.png",
        "buildings_geojson": "buildings_with_heights_1km.geojson"
    },
    "unity_axes": "SWEREF X->Unity X, SWEREF Y->Unity Z, height->Unity Y"
}

with open(meta_path, "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=2)

print("Saved:", meta_path)

Saved: /content/processed/metadata_1km.json


In [62]:
hm_check = iio.imread("/content/processed/heightmap_1km.png")
print(hm_check.shape, hm_check.dtype, int(hm_check.min()), int(hm_check.max()))

(501, 501) uint16 0 65535
